In [71]:
import sys
import os
import math
from pyspark.rdd import RDD
from pyspark import SparkConf, SparkContext
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk1.8.0_301'
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [72]:
# Mapper for Matrix M for (j, (M, i, mij))
# Mapper for Matrix N for (j, (N, k, njk))
def mapper1(line):
    matrix_index, row, col, value = line.rstrip().split(",")
    maplist = []
    #array size
    if matrix_index == 'M':
        maplist.append((int(col), (matrix_index, int(row), int(value))))
            
    return maplist

def mapper2(line):
    matrix_index, row, col, value = line.rstrip().split(",")
    maplist = []
    #array size
    if matrix_index == 'N':
        maplist.append((int(row), (matrix_index, int(col), int(value))))
            
    return maplist

In [73]:
# Mapper for ((i, k), (mij * njk))
def mapper3(x):
    # x[0] 為 key, x[1]為 value，而 value 有兩項
    # value 兩項分別為('M', 0, 1) 和 ('N', 0, 5)
    return ((x[1][0][1], x[1][1][1]), x[1][0][2]*x[1][1][2])
        

In [74]:
def reducer1(x,y):
    return x+y

In [75]:
def output(res_list):
    f = open('Outputfile.txt', 'w')
    f.write('\n'.join('{} {}'.format(x[0],x[1]) for x in res_list))
    print("Done")
        
    return

In [76]:
conf = SparkConf().setMaster("local").setAppName("matrix_multiplcation")
sc = SparkContext(conf=conf)


In [88]:
lines = sc.textFile("data/test_2x2.txt")
rddM = lines.flatMap(mapper1)
rddN = lines.flatMap(mapper2)
rddMN = rddM.join(rddN)
rddMN.take(8)

[(0, (('M', 0, 1), ('N', 0, 5))),
 (0, (('M', 0, 1), ('N', 1, 6))),
 (0, (('M', 1, 3), ('N', 0, 5))),
 (0, (('M', 1, 3), ('N', 1, 6))),
 (1, (('M', 0, 2), ('N', 0, 7))),
 (1, (('M', 0, 2), ('N', 1, 8))),
 (1, (('M', 1, 4), ('N', 0, 7))),
 (1, (('M', 1, 4), ('N', 1, 8)))]

In [89]:
rddP = rddMN.map(mapper3)
rddP.take(8)

[((0, 0), 5),
 ((0, 1), 6),
 ((1, 0), 15),
 ((1, 1), 18),
 ((0, 0), 14),
 ((0, 1), 16),
 ((1, 0), 28),
 ((1, 1), 32)]

In [91]:
listP = rddP.reduceByKey(reducer1).sortByKey()
res = listP.collect() #type(res) 為 list
# print(res)
output(res)


Done


In [70]:
sc.stop()